In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv

# Load the Cora dataset
dataset = Planetoid(root='data/Cora', name='Cora')

# Prepare data
data = dataset[0]

# Define a 2-layer GCN
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return torch.log_softmax(x, dim=1)

# Initialize model, optimizer, and loss function
model = GCN(input_dim=dataset.num_node_features, hidden_dim=16, output_dim=dataset.num_classes)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Training loop
for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

print("Training complete!")


Epoch 0, Loss: 1.9551197290420532
Epoch 10, Loss: 0.6276729702949524
Epoch 20, Loss: 0.11319617927074432
Epoch 30, Loss: 0.02560604177415371
Epoch 40, Loss: 0.009924104437232018
Epoch 50, Loss: 0.005633097141981125
Epoch 60, Loss: 0.003971583675593138
Epoch 70, Loss: 0.003149603260681033
Epoch 80, Loss: 0.0026519682724028826
Epoch 90, Loss: 0.0023040224332362413
Training complete!


In [13]:
model.eval()

with torch.no_grad():
    out = model(data)
    pred = out.argmax(dim=1) # get prediction
    
    # Calculate the accuracy on the test set
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    accuracy = int(correct) / int(data.test_mask.sum())

# Print the test accuracy
print(f'Test Accuracy: {accuracy:.4f}')

Test Accuracy: 0.7760


## Explanation:
GCN aggregates features from a node’s neighbors using graph convolutions. This allows the network to learn representations based on both node features and graph structure.
The Cora dataset is used to classify nodes into one of 7 research topics.

## Questions (1 point each):

#### 1. What would happen if we added more GCN layers (e.g., 3 layers instead of 2)? How would this affect over-smoothing?

Over-smoothing: interacting nodes would have quite similar representations; may occur b/c layers aggregate info from neighboring nodes

In [9]:
# Define a 3-layer GCN
class GCN_3_layer(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
        super(GCN_3_layer, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim1)
        self.conv2 = GCNConv(hidden_dim1, hidden_dim2)
        self.conv3 = GCNConv(hidden_dim2, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        x = self.conv3(x, edge_index)
        return torch.log_softmax(x, dim=1)

In [10]:
# Initialize model, optimizer, and loss function
model_3 = GCN_3_layer(input_dim=dataset.num_node_features, hidden_dim1=16, hidden_dim2=16, output_dim=dataset.num_classes)
optimizer_3 = optim.Adam(model_3.parameters(), lr=0.01)
criterion_3 = nn.CrossEntropyLoss()

In [11]:
for epoch in range(100):
    model_3.train()
    optimizer_3.zero_grad()

    out = model_3(data)
    loss = criterion_3(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer_3.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

print("Training complete for the 3-layer GCN!")

Epoch 0, Loss: 1.9486275911331177
Epoch 10, Loss: 0.9624478816986084
Epoch 20, Loss: 0.24392223358154297
Epoch 30, Loss: 0.020068548619747162
Epoch 40, Loss: 0.003072653664276004
Epoch 50, Loss: 0.0010185071732848883
Epoch 60, Loss: 0.0006181305507197976
Epoch 70, Loss: 0.00047641972196288407
Epoch 80, Loss: 0.0004055898170918226
Epoch 90, Loss: 0.00036247126990929246
Training complete for the 3-layer GCN!


In [12]:
model_3.eval()

with torch.no_grad(): 
    out = model_3(data)
    pred = out.argmax(dim=1) # get prediction

    # Calculate accuracy on the test set
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    accuracy = int(correct) / int(data.test_mask.sum())

print(f'Test Accuracy: {accuracy:.4f}')

Test Accuracy: 0.7630


As we add another layer to the GCN, the test accuracy drops slightly from 0.7760 to 0.7630. This suggest that adding more layers to the GCN did not improve the model's performance on the test set. 

In my opinion, this drop may be due to the over-smoothing problem, as increasing the number of the GCN layers without regularization may cause the node embeddings to become too similar to each other. This problem may lead to the model being less capable of distinguishing between nodes in different classes. This may be improved by adding regularizations like dropout.

#### 2. What would happen if we used a larger hidden dimension (e.g., 64 instead of 16)? How would this impact the model's capacity?

In [14]:
model_large_hidden = GCN(input_dim=dataset.num_node_features, hidden_dim=64, output_dim=dataset.num_classes)
optimizer_large_hidden = optim.Adam(model_large_hidden.parameters(), lr=0.01)
criterion_large_hidden = nn.CrossEntropyLoss()

In [15]:
for epoch in range(100):
    model_large_hidden.train()
    optimizer_large_hidden.zero_grad()
    
    out = model_large_hidden(data)
    loss = criterion_large_hidden(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer_large_hidden.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

print("Training complete for GCN w/ larger hidden layer!")

Epoch 0, Loss: 1.9551746845245361
Epoch 10, Loss: 0.07922256737947464
Epoch 20, Loss: 0.00296090729534626
Epoch 30, Loss: 0.0005492267664521933
Epoch 40, Loss: 0.0002485028235241771
Epoch 50, Loss: 0.00017058693629223853
Epoch 80, Loss: 0.00012135658471379429
Epoch 90, Loss: 0.00011492427438497543
Training complete for GCN w/ larger hidden layer!


In [16]:
model_large_hidden.eval()
with torch.no_grad():
    out = model_large_hidden(data)
    pred = out.argmax(dim=1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    accuracy = int(correct) / int(data.test_mask.sum())
    
print(f'Test Accuracy with larger hidden dimension: {accuracy:.4f}')

Test Accuracy with larger hidden dimension: 0.7840


The test accuracy of model with larger hidden dimension  0.7840 shows a slight improvement over both the 2-layer GCN with 16 hidden units (Test Accuracy: 0.7760) and the 3-layer GCN with 16 hidden units (Test Accuracy: 0.7630). This indicates that by increasing the number of hidden dimensions, the model gained **more capacity** to learn more complexed representations of the features, which allows the model to capture more details in the data. 

In addition, increasing the hidden dimension in this particular case does not worsen the model performance by over-smoothing or overfitting. However, there is an associated increase in computational cost as there are increased number of parameters.

#### 3. What would happen if we replaced ReLU activation with a sigmoid function? Would the performance change?

In [17]:
class GCN_sigmoid(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN_sigmoid, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.sigmoid(x)
        x = self.conv2(x, edge_index)
        return torch.log_softmax(x, dim=1)

In [18]:
model_sigmoid = GCN_sigmoid(input_dim=dataset.num_node_features, hidden_dim=16, output_dim=dataset.num_classes)
optimizer_sigmoid = optim.Adam(model_sigmoid.parameters(), lr=0.01)
criterion_sigmoid = nn.CrossEntropyLoss()

In [19]:
for epoch in range(100):
    model_sigmoid.train()
    optimizer_sigmoid.zero_grad()
    
    out = model_sigmoid(data)
    loss = criterion_sigmoid(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer_sigmoid.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')
        
print("Training complete for GCN w/ sigmoid activation!")

Epoch 0, Loss: 2.265245199203491
Epoch 10, Loss: 1.525286316871643
Epoch 20, Loss: 1.0788142681121826
Epoch 30, Loss: 0.7327616810798645
Epoch 40, Loss: 0.4851303696632385
Epoch 50, Loss: 0.3223065435886383
Epoch 60, Loss: 0.22101959586143494
Epoch 70, Loss: 0.15894536674022675
Epoch 80, Loss: 0.12027529627084732
Epoch 90, Loss: 0.09509587287902832
Training complete for GCN w/ sigmoid activation!


In [20]:
model_sigmoid.eval()
with torch.no_grad():
    out = model_sigmoid(data)
    pred = out.argmax(dim=1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    accuracy = int(correct) / int(data.test_mask.sum())
    
print(f'Test Accuracy with sigmoid activation: {accuracy:.4f}')

Test Accuracy with sigmoid activation: 0.7690


The test accuracy with sigmoid activation function is slightly lower than with ReLU. This may be attributed to the vanishing gradient problem, which slows down the learning process and lead to slight decrease in model accuracy. On the other hand, ReLu allows for more sparsity and sets some neurons to output zero, which helps prevent the vanishing gradient problem. 

Moreover, I noticed that the training time of the model with sigmoid activation is longer than the one with ReLU activation. This could be attributed to the lower computational complexity of ReLU than sigmoid.

#### 4. What would happen if we trained on only 10% of the nodes and tested on the remaining 90%? How would the performance be affected?

In [21]:
train_ratio = 0.1
num_train_nodes = int(train_ratio * data.num_nodes)

In [22]:
# Randomly shuffle the node indices (this was suggested by ChatGPT)
perm = torch.randperm(data.num_nodes)

In [23]:
train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
train_mask[perm[:num_train_nodes]] = True
test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
test_mask[perm[num_train_nodes:]] = True

In [24]:
data.train_mask = train_mask
data.test_mask = test_mask

In [25]:
model_10_pct = GCN(input_dim=dataset.num_node_features, hidden_dim=16, output_dim=dataset.num_classes)
optimizer_10_pct = optim.Adam(model_10_pct.parameters(), lr=0.01)
criterion_10_pct = nn.CrossEntropyLoss()

In [26]:
for epoch in range(100):
    model_10_pct.train()
    optimizer_10_pct.zero_grad()
    
    out = model_10_pct(data)
    loss = criterion_10_pct(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer_10_pct.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')
        
print("Training complete for GCN w/ 10% training nodes!")

Epoch 0, Loss: 1.9473978281021118
Epoch 10, Loss: 0.7699562907218933
Epoch 20, Loss: 0.20229144394397736
Epoch 30, Loss: 0.05703773722052574
Epoch 40, Loss: 0.021402880549430847
Epoch 50, Loss: 0.0108838165178895
Epoch 60, Loss: 0.007085033226758242
Epoch 70, Loss: 0.0052841054275631905
Epoch 80, Loss: 0.004255638923496008
Epoch 90, Loss: 0.003575586946681142
Training complete for GCN w/ 10% training nodes!


In [27]:
model_10_pct.eval()
with torch.no_grad():
    out = model_10_pct(data)
    pred = out.argmax(dim=1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    accuracy = int(correct) / int(data.test_mask.sum())
    
print(f'Test Accuracy with 10% training nodes: {accuracy:.4f}')

Test Accuracy with 10% training nodes: 0.8084


The test accuracy of the model trained on 10% of the nodes and test on the remaining 90% is higher than any other models constructed before. In my opinion, this could be due to several factors:

- The model may generalize better when trained on fewer nodes, because it is less likely to overfit on the training set.
- The graph structure of the dataset may be well-connected and informative enough that even a small number of training nodes is enough for the model to generalize well for the rest of the nodes.  

These hypotheses can be further validated through permutation testing, where he nodes are shuffled multiple times and different 10% nodes are used to train the model every time.

#### 5. What would happen if we used a different optimizer (e.g., RMSprop) instead of Adam? Would it affect the convergence speed?

In [28]:
data.train_mask = dataset[0].train_mask
data.test_mask = dataset[0].test_mask

In [29]:
model_rmsprop = GCN(input_dim=dataset.num_node_features, hidden_dim=16, output_dim=dataset.num_classes)
optimizer_rmsprop = optim.RMSprop(model_rmsprop.parameters(), lr=0.01)
criterion_rmsprop = nn.CrossEntropyLoss()

In [30]:
for epoch in range(100):
    model_rmsprop.train()
    optimizer_rmsprop.zero_grad()
    
    out = model_rmsprop(data)
    loss = criterion_rmsprop(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer_rmsprop.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')
        
print("Training complete for GCN w/ RMSprop optimizer!")

Epoch 0, Loss: 1.9377018213272095
Epoch 10, Loss: 0.035184767097234726
Epoch 20, Loss: 0.015747353434562683
Epoch 30, Loss: 0.009820069186389446
Epoch 40, Loss: 0.006943057291209698
Epoch 50, Loss: 0.00526403496041894
Epoch 60, Loss: 0.004169971216470003
Epoch 70, Loss: 0.003405557246878743
Epoch 80, Loss: 0.0028436449356377125
Epoch 90, Loss: 0.0024153452832251787
Training complete for GCN w/ RMSprop optimizer!


In [31]:
model_rmsprop.eval()
with torch.no_grad():
    out = model_rmsprop(data)
    pred = out.argmax(dim=1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    accuracy = int(correct) / int(data.test_mask.sum())

print(f'Test Accuracy with RMSprop optimizer: {accuracy:.4f}')

Test Accuracy with RMSprop optimizer: 0.7820


The difference between the test accuracy of the models with Adam and RMSprop is small, meaning that both optimizers perform fairly well for this task. 

In general, RMSprop tends to converge more slowly compared to Adam. This may lead to slightly less efficiency in updating weights and finding the optimal solution. (This paragraph is written with the help of ChatGPT.)

### Extra credit: 
#### 1. What would happen if we used edge weights (non-binary) in the adjacency matrix? How would it affect message passing?

In [35]:
# dir(data)

In [36]:
class GCN_with_weights(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN_with_weights, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_attr
        x = self.conv1(x, edge_index, edge_weight=edge_weight)
        x = torch.relu(x)
        x = self.conv2(x, edge_index, edge_weight=edge_weight)
        return torch.log_softmax(x, dim=1)

In [37]:
model_weighted = GCN_with_weights(input_dim=dataset.num_node_features, hidden_dim=16, output_dim=dataset.num_classes)
optimizer_weighted = optim.Adam(model_weighted.parameters(), lr=0.01)
criterion_weighted = nn.CrossEntropyLoss()

In [38]:
for epoch in range(100):
    model_weighted.train()
    optimizer_weighted.zero_grad()
    
    out = model_weighted(data)
    loss = criterion_weighted(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer_weighted.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')
        
print("Training complete for GCN w/ edge weights!")

Epoch 0, Loss: 1.9426648616790771
Epoch 10, Loss: 0.5306446552276611
Epoch 20, Loss: 0.08349411189556122
Epoch 30, Loss: 0.017982272431254387
Epoch 40, Loss: 0.006782583426684141
Epoch 50, Loss: 0.003893067594617605
Epoch 60, Loss: 0.0028360977303236723
Epoch 70, Loss: 0.0023138488177210093
Epoch 80, Loss: 0.0019932996947318316
Epoch 90, Loss: 0.0017628786154091358
Training complete for GCN w/ edge weights!


In [39]:
model_weighted.eval()
with torch.no_grad():
    out = model_weighted(data)
    pred = out.argmax(dim=1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    accuracy = int(correct) / int(data.test_mask.sum())

print(f'Test Accuracy with edge weights: {accuracy:.4f}')

Test Accuracy with edge weights: 0.7880


When we use binary adjacency matrices in GCNs, the information passed between nodes is binary, meaning there is either a connection (1) or no connection (0). On the other hand, if we use edge weights (non-binary) in the adjacency matrix, the information aggregated by each node from its neighbors will be weighted by the strength of the connection between nodes. This means that during the message passing process, nodes connected by stronger edges would contribute more to the final representation of a node. 

By using the edge weight in adjacency matrix, the model may capture more details about the relationships between nodes and better learn the features, leading to richer node embeddings and potentially improving the model performance. This may also introduce more complexity in the model learning process, which can be suitable for complex network or graph structures but also lead to higher computational cost.

#### 2. What would happen if we removed the log-softmax function in the output layer? Would the loss function still work correctly?

In [2]:
class GCN_no_logsoftmax(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN_no_logsoftmax, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index) 
        return x 

In [3]:
model_no_logsoftmax = GCN_no_logsoftmax(input_dim=dataset.num_node_features, hidden_dim=16, output_dim=dataset.num_classes)
optimizer_no_logsoftmax = optim.Adam(model_no_logsoftmax.parameters(), lr=0.01)
criterion_no_logsoftmax = nn.CrossEntropyLoss()  # This already applies softmax internally

In [7]:
for epoch in range(100):
    model_no_logsoftmax.train()
    optimizer_no_logsoftmax.zero_grad()
    
    out = model_no_logsoftmax(data)
    
    loss = criterion_no_logsoftmax(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer_no_logsoftmax.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

print("Training complete for GCN w/o log softmax!")

Epoch 0, Loss: 1.953733205795288
Epoch 10, Loss: 0.5637298226356506
Epoch 20, Loss: 0.08987681567668915
Epoch 30, Loss: 0.01918867975473404
Epoch 40, Loss: 0.007057531271129847
Epoch 50, Loss: 0.0039271279238164425
Epoch 60, Loss: 0.0028138982597738504
Epoch 70, Loss: 0.002280070213600993
Epoch 80, Loss: 0.0019572952296584845
Epoch 90, Loss: 0.0017287489026784897
Training complete for GCN w/o log softmax!


In [10]:
import torch.nn.functional as F

In [12]:
model_no_logsoftmax.eval()
with torch.no_grad():
    out = model_no_logsoftmax(data)

# Print the output for a few nodes
print("Output (first 5 training nodes):")
print(out[data.train_mask][:5])

# Apply softmax to the output to convert them into probabilities
probabilities = F.softmax(out, dim=1)

# Print the probabilities for the same nodes
print("\nProbabilities (after applying softmax, first 5 training nodes):")
print(probabilities[data.train_mask][:5])

# Check the sum of probabilities
print("\nSum of probabilities:")
print(probabilities[data.train_mask][:5].sum(dim=1))

Output (first 5 training nodes):
tensor([[-3.2499, -1.0346, -4.3134,  6.3326, -2.1912, -1.8376, -5.6061],
        [-4.8734, -3.7940,  0.3320, -3.4253, 11.6896, -4.2241, -4.5810],
        [-6.3840, -3.6482, -0.7484,  0.0278,  9.3596, -3.5577, -5.5921],
        [ 7.7711, -3.0735, -2.5449, -4.2608, -3.7189, -3.7195, -0.4565],
        [-3.7300, -1.3807, -2.6438,  5.7616, -2.0244, -1.7527, -6.1350]])

Probabilities (after applying softmax, first 5 training nodes):
tensor([[6.8841e-05, 6.3083e-04, 2.3765e-05, 9.9879e-01, 1.9843e-04, 2.8262e-04,
         6.5244e-06],
        [6.4088e-08, 1.8861e-07, 1.1680e-05, 2.7269e-07, 9.9999e-01, 1.2268e-07,
         8.5853e-08],
        [1.4540e-07, 2.2425e-06, 4.0750e-05, 8.8557e-05, 9.9987e-01, 2.4549e-06,
         3.2101e-07],
        [9.9965e-01, 1.9503e-05, 3.3089e-05, 5.9493e-06, 1.0228e-05, 1.0223e-05,
         2.6709e-04],
        [7.5328e-05, 7.8930e-04, 2.2320e-04, 9.9795e-01, 4.1466e-04, 5.4411e-04,
         6.7994e-06]])

Sum of probabilitie

If the log-softmax function is removed from the output layer of the model, the model will output logits, which is the raw scores each class. When we apply softmax manually on these numbers and convert them into probabilities, they sum up to one for each node. 

In [13]:
train_nodes = data.train_mask.nonzero(as_tuple=True)[0][:5]

# Manually calculate the loss w/ softmax and NLL
log_probs = F.log_softmax(out[train_nodes], dim=1)  # Log-softmax
nll_loss_manual = -log_probs[range(5), data.y[train_nodes]].mean()  # NLL

print(f"Manually calculated NLL loss: {nll_loss_manual.item()}")

# Compare with nn.CrossEntropyLoss
nll_loss_crossentropy = criterion_no_logsoftmax(out[train_nodes], data.y[train_nodes])

print(f"Loss from nn.CrossEntropyLoss: {nll_loss_crossentropy.item()}")

Manually calculated NLL loss: 0.0007521277293562889
Loss from nn.CrossEntropyLoss: 0.0007521277293562889


After manually computing the negative log-likelihood loss from the output logits and comparing them with the output of CrossEntropyLoss, they are the same. This indicates that `nn.CrossEntropyLoss` will internally apply the softmax function to convert the logits into probabilities before computing the loss. Therefore, the loss function will still run correctly even if we remove softmax from the model output layer. 

## No points, just for you to think about:

#### 1. What would happen if we applied dropout to the node features during training? How would it affect the model’s generalization?

#### 2. What would happen if we used mean-pooling instead of summing the messages in the GCN layers?


#### 3. What would happen if we pre-trained the node features using a different algorithm, like Node2Vec, before feeding them into the GCN?